In [1]:
import requests
from dataclasses import dataclass
import json

with open("config.json") as config_file:
    config = json.load(config_file)
league_url = "http://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/" + str(config['league_id'])
cookies = config['cookies']

In [39]:
from api import ESPN_API

In [40]:
espn_api = ESPN_API()

In [11]:
random_url = "https://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/5192152?view=mMatchup&view=mMatchupScore&view=mRoster&view=mScoreboard&view=mTeam"

In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
from classes import Player, Team

In [43]:
teams = espn_api.get_teams()

In [8]:
Player.__dataclass_fields__.keys()

dict_keys(['pid', 'name', 'status', 'stats', 'team_id'])

In [12]:
matchup = "?view=mMatchup"
matchup_json = requests.get(league_url, cookies=cookies, params = {"view":"mMatchup"}).json()

In [13]:
matchup_json.keys()

dict_keys(['draftDetail', 'gameId', 'id', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'status', 'teams'])

In [15]:
matchup_json['gameId']

3

In [17]:
matchup_json['teams'][0]

{'id': 1,
 'roster': {'entries': [{'acquisitionDate': 1608520380203,
    'acquisitionType': 'DRAFT',
    'injuryStatus': 'NORMAL',
    'lineupSlotId': 3,
    'pendingTransactionIds': None,
    'playerId': 6583,
    'playerPoolEntry': {'id': 6583,
     'keeperValue': 1,
     'keeperValueFuture': 1,
     'lineupLocked': True,
     'onTeamId': 1,
     'player': {'active': True,
      'defaultPositionId': 4,
      'droppable': False,
      'eligibleSlots': [3, 4, 6, 8, 9, 10, 11, 12, 13],
      'firstName': 'Anthony',
      'fullName': 'Anthony Davis',
      'id': 6583,
      'injured': False,
      'injuryStatus': 'ACTIVE',
      'lastName': 'Davis',
      'lastNewsDate': 1609391237000,
      'lastVideoDate': 1607149672000,
      'proTeamId': 13,
      'universeId': 2},
     'rosterLocked': True,
     'status': 'ONTEAM',
     'tradeLocked': False},
    'status': 'NORMAL'},
   {'acquisitionDate': 1608520380204,
    'acquisitionType': 'DRAFT',
    'injuryStatus': 'NORMAL',
    'lineupSlotId

In [26]:
home_view = "?rosterForTeamId=9&view=mDraftDetail&view=mLiveScoring&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mRoster&view=mSettings&view=mTeam&view=modular&view=mNav"
home = requests.get(league_url+home_view).json()




In [27]:
home.keys()

dict_keys(['draftDetail', 'gameId', 'id', 'members', 'pendingTransactions', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'settings', 'status', 'teams'])

In [64]:
scoreboard_view = "?view=mMatchup&view=mMatchupScore&view=mRoster&view=mScoreboard&view=mSettings&view=mStatus&view=mTeam&view=modular&view=mNav"
scoreboard = requests.get(league_url+scoreboard_view).json()

In [65]:
scoreboard.keys()

dict_keys(['draftDetail', 'gameId', 'id', 'members', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'settings', 'status', 'teams'])

In [73]:
schedule = scoreboard['schedule']

In [75]:
schedule[0].keys()

dict_keys(['away', 'home', 'id', 'matchupPeriodId', 'playoffTierType', 'winner'])

In [12]:
# for obj in schedule:
#     print(obj['matchupPeriodId'])

In [13]:
teams[5]

KeyError: 5

In [77]:
schedule[0]['home']

{'adjustment': 0.0,
 'cumulativeScore': {'losses': 4,
  'scoreByStat': {'0': {'ineligible': False,
    'rank': 0.0,
    'result': 'LOSS',
    'score': 347.0},
   '1': {'ineligible': False, 'rank': 0.0, 'result': 'WIN', 'score': 41.0},
   '2': {'ineligible': False, 'rank': 0.0, 'result': 'WIN', 'score': 35.0},
   '3': {'ineligible': False, 'rank': 0.0, 'result': 'LOSS', 'score': 78.0},
   '6': {'ineligible': False, 'rank': 0.0, 'result': 'WIN', 'score': 155.0},
   '42': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 27.0},
   '11': {'ineligible': False, 'rank': 0.0, 'result': 'WIN', 'score': 52.0},
   '13': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 120.0},
   '14': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 274.0},
   '15': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 75.0},
   '16': {'ineligible': False, 'rank': 0.0, 'result': None, 'score': 91.0},
   '17': {'ineligible': False, 'rank': 0.0, 'result': 'LOSS', 'score': 32.0}

In [59]:
from typing import Dict
@dataclass
class Team:
    team_id: int
    owner: str
    abbrev: str
    name: str
    season_totals: Dict[str, float]

In [60]:
teams = {}
for team in home['teams']:
    curr_id = team['id']
    team_obj = Team(team_id = team['id'],
                owner = team['primaryOwner'][1:-1],
                abbrev = team['abbrev'],
                name = team['nickname'],
                totals = team['valuesByStat'])
    teams[curr_id] = team_obj

In [255]:
info = requests.get(league_url, 
                    cookies=cookies,
                    params={"view":"kona_player_info"},
                   headers={'x-fantasy-filter':
                           '{"players":{"filterStatus":{"value":["FREEAGENT","WAIVERS"]},"filterSlotIds":{"value":[0,1,2,3,4,5,6,7,8,9,10,11]},"filterRanksForScoringPeriodIds":{"value":[12]},"limit":250,"sortPercOwned":{"sortAsc":false,"sortPriority":1},"sortDraftRanks":{"sortPriority":100,"sortAsc":true,"value":"STANDARD"},"filterRanksForRankTypes":{"value":["STANDARD"]},"filterStatsForTopScoringPeriodIds":{"value":5,"additionalValue":["002021","102021","002020","012021","022021","032021","042021"]}}}',
                           'x-fantasy-platform': 'kona-PROD-a5abcf16cafe5c335041277beeafcabc2a236402',
                           'x-fantasy-source': 'kona'}
                   ).json()
for player in info['players']:
    if player['player']['lastName'] == "Barrett":
        print('sup')

sup


In [362]:
from typing import Optional

@dataclass
class Stat:
    points: float
    turnovers: float
    minutes: float
    fgm: float
    fga: float
    ftm: float
    fta: float
    made_threes: float
    attempted_threes: float
    off_rebounds: float
    def_rebounds: float
    rebounds: float
    assists: float
    steals: float
    blocks: float
    fouls: float
        
    @property
    def fg_pct(self) -> float:
        return self.fgm/self.fga

    @property
    def ft_pct(self) -> float:
        return self.ftm/self.fta
    
    @staticmethod
    def parse_stat(stat_obj):
        def from_stat(index):
            return stat_obj[index] if index in stat_obj else None

        return Stat(
            points = from_stat('0'),
            blocks = from_stat('1'),
            steals = from_stat('2'),
            assists = from_stat('3'),
            off_rebounds = from_stat('4'),
            def_rebounds = from_stat('5'),
            rebounds = from_stat('6'),
            fouls = from_stat('9'),
            turnovers = from_stat('11'),
            fgm = from_stat('13'),
            fga = from_stat('14'),
            ftm = from_stat('15'),
            fta = from_stat('16'),
            made_threes = from_stat('17'),
            attempted_threes = from_stat('18'),
            minutes = from_stat('40')
        )

        # 7,8 = ?
        # fg_pct = 19
        # ft_pct = 20
        # 3_pct = 21
        # efg = 22
    
@dataclass
class StatDict:
    avg: Stat
    total: Stat
        
@dataclass
class Stats:
    last7: StatDict
    last15: StatDict
    last30: StatDict
    s20: StatDict
    s21: StatDict
    s21_proj: StatDict

    @stt
@dataclass
class Player:
    pid: int
    name: str
    status: str # FREEAGENT, WAIVERS, etc.
    stats: Stats

In [379]:
# def get_stats(stats):
#     def get_stat(stat_id):
#         stat_objs = [stat for stat in stats if stat['id'] == stat_id]
#         if not stat_objs:
#             return None
#         stat_obj = stat_objs[0]
#         return StatDict(avg = Stat.parse_stat(stat_obj['averageStats']) if 'averageStats' in stat_obj else None, 
#                         total= Stat.parse_stat(stat_obj['stats']))
#     return Stats(
#         last7 = get_stat('012021'),
#         last15 = get_stat('022021'),
#         last30 = get_stat('032021'),
#         s20 = get_stat('002020'),
#         s21 = get_stat('002021'),
#         s21_proj = get_stat('102021')
#     )


In [50]:
fantasy_filter = {
    "players":
        {"filterStatus": {"value": ["FREEAGENT","WAIVERS"]},
         "filterSlotIds":{"value":[0,1,2,3,4,5,6,7,8,9,10,11]},
         "filterRanksForScoringPeriodIds":{"value":[12]},
         "limit":1000,
         "sortPercOwned":{"sortAsc":"false","sortPriority":1},
         "sortDraftRanks":{"sortPriority":100,"sortAsc":"true","value":"STANDARD"},
         "filterRanksForRankTypes":{"value":["STANDARD"]},
         "filterStatsForTopScoringPeriodIds":{"value":5,"additionalValue":["002021","102021","002020","012021","022021","032021","042021"]}
        }
}

In [51]:
info = requests.get(league_url, 
    cookies=cookies,
    params={"view":"kona_player_info"},
   headers={'x-fantasy-filter': json.dumps(fantasy_filter),
           'x-fantasy-platform': 'kona-PROD-a5abcf16cafe5c335041277beeafcabc2a236402',
           'x-fantasy-source': 'kona'}
   ).json()

In [49]:
for stat in player['player']['stats']:
    print(stat)

NameError: name 'player' is not defined

In [54]:
player['player']['proTeamId']

18

In [55]:
player['onTeamId']

0

In [52]:
free_agents = {}
name_to_pid = {}
for player in info['players']:
    player_obj = player['player']
    free_agents[player['id']] = Player(
        pid = player['id'],
        name = player_obj['fullName'],
        status = player['status'],
        stats = get_stats(player_obj['stats'])
    )
    name_to_pid[player_obj['fullName']] = player['id']


NameError: name 'get_stats' is not defined

In [56]:
players, name_to_pid = espn_api.get_player_info()

In [57]:
players[name_to_pid] = [
]

1019

In [388]:
free_agents[4395625]#()

Player(pid=4395625, name='RJ Barrett', status='WAIVERS', stats=Stats(last7=StatDict(avg=Stat(points=12.75, turnovers=2.75, minutes=37.75, fgm=4.25, fga=16.5, ftm=4.25, fta=5.25, made_threes=0.0, attempted_threes=5.25, off_rebounds=1.5, def_rebounds=5.75, rebounds=7.25, assists=3.25, steals=1.0, blocks=0.0, fouls=2.0), total=Stat(points=51.0, turnovers=11.0, minutes=151.0, fgm=17.0, fga=66.0, ftm=17.0, fta=21.0, made_threes=0.0, attempted_threes=21.0, off_rebounds=6.0, def_rebounds=23.0, rebounds=29.0, assists=13.0, steals=4.0, blocks=0.0, fouls=8.0)), last15=StatDict(avg=Stat(points=15.4, turnovers=2.4, minutes=37.0, fgm=5.6, fga=16.2, ftm=3.6, fta=4.6, made_threes=0.6, attempted_threes=4.8, off_rebounds=1.2, def_rebounds=6.2, rebounds=7.4, assists=3.6, steals=0.8, blocks=0.0, fouls=2.2), total=Stat(points=77.0, turnovers=12.0, minutes=185.0, fgm=28.0, fga=81.0, ftm=18.0, fta=23.0, made_threes=3.0, attempted_threes=24.0, off_rebounds=6.0, def_rebounds=31.0, rebounds=37.0, assists=18.0,

In [360]:
get_stats(barrett_stats).last7.avg

Stat(points=12.75, turnovers=2.75, minutes=37.75, fgm=4.25, fga=16.5, ftm=4.25, fta=5.25, made_threes=0.0, attempted_threes=5.25, off_rebounds=1.5, def_rebounds=5.75, rebounds=7.25, assists=3.25, steals=1.0, blocks=0.0, fouls=2.0)

In [345]:
# for player in info['players']:
# #     obj = player['player']
# # #     print(obj['firstName']+' '+obj['lastName'])
# #     stats = obj['stats']
# #     if obj['lastName'] == 'Barrett':
# #         barrett_stats = stats
# #     elif obj['lastName'] == 'Hunter':
# #         hunter_stats = stats
# for stat in barrett_stats:
#     print(stat['externalId'], stat['id'], stat['stats'])
#     print('--------------')

In [261]:
player['id']

2184

In [93]:
agents['schedule']

[{'away': {'adjustment': 0.0,
   'cumulativeScore': {'losses': 5,
    'statBySlot': {'15': {'exceededOnScoringPeriod': 6,
      'limitExceeded': True,
      'statId': 42,
      'value': 27.0}},
    'ties': 0,
    'wins': 4},
   'teamId': 7,
   'tiebreak': 0.0,
   'totalPoints': 0.0},
  'home': {'adjustment': 0.0,
   'cumulativeScore': {'losses': 4,
    'statBySlot': {'15': {'exceededOnScoringPeriod': 6,
      'limitExceeded': True,
      'statId': 42,
      'value': 27.0}},
    'ties': 0,
    'wins': 5},
   'teamId': 5,
   'tiebreak': 0.0,
   'totalPoints': 0.0},
  'id': 1,
  'matchupPeriodId': 1,
  'playoffTierType': 'NONE',
  'winner': 'HOME'},
 {'away': {'adjustment': 0.0,
   'cumulativeScore': {'losses': 4,
    'statBySlot': {'15': {'exceededOnScoringPeriod': 0,
      'limitExceeded': False,
      'statId': 42,
      'value': 23.0}},
    'ties': 1,
    'wins': 4},
   'teamId': 9,
   'tiebreak': 0.0,
   'totalPoints': 0.0},
  'home': {'adjustment': 0.0,
   'cumulativeScore': {'losse